In [ ]:
# Creem la conex amb la db MYSQL
import mysql.connector

db = mysql.connector.connect(
    host="localhost",
    username="root",
    password="123321",
    database="tenfe"
)

In [ ]:
import pandas as pd

In [ ]:
# Llegim el excel i el guardem en un dataframe
df = pd.read_excel('../data/linies/R1.xlsx')

In [ ]:
# Creem un cursor per poder fer les consultes
cursor = db.cursor()

In [ ]:
# Extraiem el ID_PARADA del dataframe i el guardem en una llista
parades = df['ID_PARADA'].unique()

parades

# Inserim les parades a la taula PARADES
for parada in parades:
    # Comprovem que no existeixi la parada
    cursor.execute("SELECT * FROM parades WHERE nom = %s", (parada,))
    result = cursor.fetchone()
    if result:
        continue
    # Inserim la parada
    sql = "INSERT INTO parades (nom) VALUES (%s)"
    val = (parada,)
    cursor.execute(sql, val)


# Guardem els canvis
db.commit()

In [ ]:
# Canviem del df cada nom de parada pel seu id
for _, fila in df.iterrows():
    sql = "SELECT id_parada FROM parades WHERE nom = %s"
    val = (fila['ID_PARADA'],)
    cursor.execute(sql, val)
    result = cursor.fetchone()
    # Canviem de la fila el nom de la parada pel seu id
    try:
        df.loc[df['ID_PARADA'] == fila['ID_PARADA'], 'ID_PARADA'] = result[0]
    except:
        print("error en la fila: ", fila)

df

In [ ]:
# A tota la columna HORA la passem a string i posteriorment li treiem els ultims 3 caracters
df['HORA'] = df['HORA'].astype(str)
df['HORA'] = df['HORA'].str[:-3]

In [ ]:
# Creem la linia a la taula linies si no existeix
for _, fila in df.iterrows():
    # Comprovem que no existeixi la linia
    cursor.execute("SELECT * FROM linies WHERE nom = %s", (fila['LINIA'],))
    result = cursor.fetchone()
    if result:
        continue
    # Inserim la linia
    sql = "INSERT INTO linies (nom) VALUES (%s)"
    val = (fila['LINIA'],)
    cursor.execute(sql, val)

In [ ]:
# Inserim les dades de l'excel a la db
for _, row in df.iterrows():
    # Abans que res comprovem que no existeixi el registre
    cursor.execute("SELECT * FROM horarios WHERE id_parada = %s AND hora = %s AND id_viaje = %s AND orden = %s", (row['ID_PARADA'], row['HORA'], row['ID_VIAJE'], row['ORDEN']))
    result = cursor.fetchone()
    if result:
        continue
    # Inserim el registre
    sql = "INSERT INTO horarios (linia, id_parada, hora, id_viaje, orden) VALUES (%s, %s, %s, %s, %s)"
    val = (row['LINIA'], row['ID_PARADA'], row['HORA'], row['ID_VIAJE'], row['ORDEN'])
    cursor.execute(sql, val)
    
db.commit()